Este script lo utilizo para recuperar el histórico desde la app

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd

In [4]:
# Request users and pass
users = "e2varadero@gmail.com"
password = "xxx"
url = "https://liga-profesional-de-aficionados-al-futbol.sporteasy.net/stats/all/player/"


In [12]:
# Inicializar selenium
from selenium import webdriver

# Abrir navegador
driver = webdriver.Chrome()

# abrir url
driver.get(url)

Este bloque recupera las stats de cada JORNADA desde APP. 
- Se recuperan más campos para hacer el histórico

In [ ]:
TABLE_CLASS = "_LSFb4IwrZAI41Kk9tgr"

NAME_ROWS_CLASS_SOUP = "div.fCdW_eIEyEWMQ_Y4Xjid"
NAME_TEXT_CLASS_SOUP = "a.NznEGSuhjoTFcwzPHzNd"

STATS_ROWS_CLASS_SOUP = "div.RYFw3QcNNvVNj3bsW8gM"
STATS_VALUE_CLASS_SOUP = "div.nohm4uA0k9j71NMyNuzL"

STAR_NAME_CLASS = "Ge68L6q7HYFPnKWBVagq"
STAR_VOTE_CLASS = "wgJVUNsyjXu6Uxw905ah"


# Nombre
# ----------------------------------------------------------
tables = driver.find_elements(By.CLASS_NAME, TABLE_CLASS)
name_table = tables[0]
html_content = name_table.get_attribute('outerHTML')
# BS
soup = BeautifulSoup(html_content, 'html.parser')

# Seleccionamos filas de tabla nombre
name_rows = soup.select(NAME_ROWS_CLASS_SOUP)
# Extraemos Datos (nombres)
names = []
data = []
for row in name_rows:
    name = row.select_one(NAME_TEXT_CLASS_SOUP).text
    names.append(name)

# Stats
# ----------------------------------------------------------
stats_table = tables[1]
html_content = stats_table.get_attribute('outerHTML')
# BS
soup = BeautifulSoup(html_content, 'html.parser')

# seleccionamos filas de la tabla stats
stats_rows = soup.select(STATS_ROWS_CLASS_SOUP)
# Extraemos Datos (stats)
for row in stats_rows:
    columns = row.select(STATS_VALUE_CLASS_SOUP)  # Selector para las celdas
    values = [col.text.strip() for col in columns]
    data.append(values)

# Extraer estrella del partido
star_name = driver.find_element(By.CLASS_NAME, STAR_NAME_CLASS).text.strip()
star_votes = driver.find_element(By.CLASS_NAME, STAR_VOTE_CLASS).text
votes = star_votes.replace('Votos', '').split('/')[0].strip()
total_votes = star_votes.replace('Votos', '').split('/')[1].strip()

# Convertir los datos a un DataFrame para un manejo más cómodo
df = pd.DataFrame(data, columns=['media', 'my_note', 'p','goles', 'asistencias', 'amarillas', 'rojas', 't', 'm'])
df['nombre'] = names
df['mvp'] = 0
df['votes'] = 0
df['total_votes'] = total_votes
df.loc[df['nombre'] == star_name, 'mvp'] = 1
df.loc[df['nombre'] == star_name, 'votes'] = int(votes)

# quitamos a los que no participan (goles = '-')
df = df[df['goles'] != '-'].copy()

df = df[['nombre', 'goles', 'asistencias', 'media', 'mvp', 'amarillas', 'rojas', 'votes', 'total_votes']]
df.columns = ['name', 'goals', 'assists', 'media', 'mvp', 'yellow_card', 'red_card', 'votes', 'total_votes']
df

# obtener player_id


# TODO Agregar la columna de equipo, me parece que es la que dice min. Pero en ese caso no hay colores.
# Esto solo se necesitaría aclarar en el histórico, ya que en las jornadas que hay excel (todas las siguiente) se saca
# el equipo del mismo

,name,goals,assists,media,mvp,yellow_card,red_card,votes,total_votes
0,Della Invitado,0,1,,0,0,0,0,17
1,João Paulo Vitor,2,0,,0,0,0,0,17
2,Tito Marcano,1,0,,0,0,0,0,17
3,Vicente Comes,0,2,,0,0,0,0,17
4,Samuel Rojas,0,0,,0,0,0,0,17
5,Ricardo Pereira,1,0,,0,0,0,0,17
6,Roman Diaz,2,0,,0,0,0,0,17
7,Nico Riccelli,0,0,,0,0,0,0,17
8,Miguel De Abreu,0,1,,1,0,0,9,17
9,Mauricio Barroso,0,0,,0,0,0,0,17


Este bloque es para recuperar stats de las tablas generales de cada Season

In [91]:
# get table html by class
table = driver.find_element(By.ID, "StatsTable")
html_content = table.get_attribute('outerHTML')
# Crear un objeto BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Encontrar todas las filas de la tabla
rows = soup.select('div.kpMSHj4SLGAzDaYFdjmH')
# Extraer datos de las filas
nombres = []
data = []
for row in rows:
    # extrae el nombre si la clase es wt_HGZW7AthsUxC4oNIp
    if row.select_one("a.X2mdWGflgDTSG73l82th"):
        name = row.select_one("a.X2mdWGflgDTSG73l82th").text
        nombres.append(name)
        continue
    columns = row.select('div.Value')  # Selector para las celdas

    values = [col.text.strip() for col in columns]
    data.append(values)

# Convertir los datos a un DataFrame para un manejo más cómodo
df = pd.DataFrame(data, columns=['juegos', 'participacion', 'e', 'p', 'goles', 'asistencias', 'mvp', 'media', 'media_max', 'media_min', 'amarillas', 'rojas'])
df['nombre'] = nombres

df = df[['nombre', 'juegos', 'participacion', 'goles', 'asistencias', 'mvp', 'media', 'media_max', 'media_min', 'amarillas', 'rojas']]


In [92]:
df.head(5)

,nombre,juegos,participacion,goles,asistencias,mvp,media,media_max,media_min,amarillas,rojas
0,Emanuel Pereira,59,49,27,44,1,"7,0","8,4","6,0",1,0
1,Arturo Da Silva,58,49,99,59,9,"7,1","9,2","5,1",0,0
2,Diego Oliveros,55,50,66,57,3,"6,6","8,9","5,3",0,1
3,Miguel De Abreu,55,45,31,24,2,"6,9","8,6","6,0",0,0
4,A. Diaz Díaz Reyna,53,43,31,43,1,"7,0","8,6","5,5",0,0


In [93]:
df.dropna(how='all').reset_index(drop=True)
df.to_excel("data/total.xlsx")